In [91]:
import pickle
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [92]:
temporal_walks_file = '../datasets/collegemsg/temporal_walks.pkl'

with open(temporal_walks_file, 'rb') as file:
    temporal_walks = pickle.load(file)

In [114]:
#word2vec model training

embed_dim = 512
window = 10
min_count = 1
workers = 4

w2v = Word2Vec(sentences = temporal_walks, 
              vector_size = embed_dim, 
              window = window, 
              min_count = min_count, 
              workers = workers)

In [115]:
# evalution
pos_edges_file = '../datasets/collegemsg/positive_edges.pkl'
neg_edges_file = '../datasets/collegemsg/negative_edges.pkl'

with open(pos_edges_file, 'rb') as pfile, open(neg_edges_file, 'rb') as nfile:
    pos_edges = pickle.load(pfile)
    neg_edges = pickle.load(nfile)

In [116]:
eval_dataX = []
eval_datay = []
keys = w2v.wv.key_to_index.keys()

for positive, negative in list(zip(pos_edges, neg_edges)):
    if positive[0] in keys and negative[0] in keys and positive[1] in keys and negative[1] in keys:
        eval_dataX.append((w2v.wv.get_vector(positive[0]) + w2v.wv.get_vector(positive[1]))/2)
        eval_dataX.append((w2v.wv.get_vector(negative[0]) + w2v.wv.get_vector(negative[1]))/2)
        eval_datay.extend([1,0])

In [117]:
trainX, testX, trainy, testy = train_test_split(eval_dataX, eval_datay, test_size = 0.1, stratify = eval_datay)

In [118]:
model = LogisticRegression()
model.fit(trainX, trainy)

/home/machine-x/miniconda3/envs/torch/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [119]:
preds = model.predict(testX)

In [120]:
accuracy_score(preds, testy)

0.6549227799227799